# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 3:34PM,242260,19,DEX0007211,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jul 19 2022 3:34PM,242260,19,DEX0007212,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jul 19 2022 3:34PM,242260,19,DEX0007213,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jul 19 2022 3:28PM,242259,16,8292757,Sartorius Bioprocess Solutions,Released
4,Jul 19 2022 3:28PM,242259,16,8295402,Sartorius Bioprocess Solutions,Released
5,Jul 19 2022 3:28PM,242258,10,Enova-10325,Emerginnova,Released
6,Jul 19 2022 3:28PM,242258,10,Enova-10326,Emerginnova,Released
7,Jul 19 2022 3:16PM,242255,10,0085889480,ISDIN Corporation,Released
8,Jul 19 2022 3:16PM,242255,10,0085889642,ISDIN Corporation,Released
9,Jul 19 2022 3:16PM,242255,10,0085889658,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,242255,Released,12
40,242256,Released,1
41,242258,Released,2
42,242259,Released,2
43,242260,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242255,NaN,NaN,12.0
242256,NaN,NaN,1.0
242258,NaN,NaN,2.0
242259,NaN,NaN,2.0
242260,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242147,0.0,0.0,1.0
242150,17.0,3.0,4.0
242160,0.0,0.0,1.0
242168,0.0,4.0,1.0
242173,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242147,0,0,1
242150,17,3,4
242160,0,0,1
242168,0,4,1
242173,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242147,0,0,1
1,242150,17,3,4
2,242160,0,0,1
3,242168,0,4,1
4,242173,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242147,,,1
1,242150,17,3,4
2,242160,,,1
3,242168,,4,1
4,242173,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 3:34PM,242260,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
3,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions
5,Jul 19 2022 3:28PM,242258,10,Emerginnova
7,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation
19,Jul 19 2022 3:15PM,242256,21,"NBTY Global, Inc."
20,Jul 19 2022 3:02PM,242253,10,"Snap Medical Industries, LLC"
21,Jul 19 2022 3:01PM,242252,19,"GCH Granules USA, Inc."
22,Jul 19 2022 3:01PM,242251,10,"Snap Medical Industries, LLC"
23,Jul 19 2022 2:55PM,242249,10,"Citieffe, Inc."
24,Jul 19 2022 2:54PM,242248,10,"E5 Pharma, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 3:34PM,242260,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,3
1,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,,,2
2,Jul 19 2022 3:28PM,242258,10,Emerginnova,,,2
3,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,,,12
4,Jul 19 2022 3:15PM,242256,21,"NBTY Global, Inc.",,,1
5,Jul 19 2022 3:02PM,242253,10,"Snap Medical Industries, LLC",,,1
6,Jul 19 2022 3:01PM,242252,19,"GCH Granules USA, Inc.",,,1
7,Jul 19 2022 3:01PM,242251,10,"Snap Medical Industries, LLC",,,1
8,Jul 19 2022 2:55PM,242249,10,"Citieffe, Inc.",,,1
9,Jul 19 2022 2:54PM,242248,10,"E5 Pharma, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 3:34PM,242260,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
1,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,2,,
2,Jul 19 2022 3:28PM,242258,10,Emerginnova,2,,
3,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,12,,
4,Jul 19 2022 3:15PM,242256,21,"NBTY Global, Inc.",1,,
5,Jul 19 2022 3:02PM,242253,10,"Snap Medical Industries, LLC",1,,
6,Jul 19 2022 3:01PM,242252,19,"GCH Granules USA, Inc.",1,,
7,Jul 19 2022 3:01PM,242251,10,"Snap Medical Industries, LLC",1,,
8,Jul 19 2022 2:55PM,242249,10,"Citieffe, Inc.",1,,
9,Jul 19 2022 2:54PM,242248,10,"E5 Pharma, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 3:34PM,242260,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
1,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,2,,
2,Jul 19 2022 3:28PM,242258,10,Emerginnova,2,,
3,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,12,,
4,Jul 19 2022 3:15PM,242256,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 3:34PM,242260,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,NaN,NaN
1,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
2,Jul 19 2022 3:28PM,242258,10,Emerginnova,2.0,NaN,NaN
3,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,12.0,NaN,NaN
4,Jul 19 2022 3:15PM,242256,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 3:34PM,242260,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,0.0,0.0
1,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
2,Jul 19 2022 3:28PM,242258,10,Emerginnova,2.0,0.0,0.0
3,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,12.0,0.0,0.0
4,Jul 19 2022 3:15PM,242256,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3875723,30.0,9.0,0.0
12,242230,0.0,2.0,0.0
15,484346,5.0,3.0,17.0
16,242259,2.0,0.0,0.0
18,484441,2.0,0.0,0.0
19,1695629,33.0,10.0,0.0
20,968861,36.0,1.0,0.0
21,726563,3.0,0.0,0.0
22,484422,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3875723,30.0,9.0,0.0
1,12,242230,0.0,2.0,0.0
2,15,484346,5.0,3.0,17.0
3,16,242259,2.0,0.0,0.0
4,18,484441,2.0,0.0,0.0
5,19,1695629,33.0,10.0,0.0
6,20,968861,36.0,1.0,0.0
7,21,726563,3.0,0.0,0.0
8,22,484422,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,30.0,9.0,0.0
1,12,0.0,2.0,0.0
2,15,5.0,3.0,17.0
3,16,2.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,33.0,10.0,0.0
6,20,36.0,1.0,0.0
7,21,3.0,0.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,12,Released,0.0
2,15,Released,5.0
3,16,Released,2.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,2.0,3.0,0.0,0.0,10.0,1.0,0.0,0.0
Released,30.0,0.0,5.0,2.0,2.0,33.0,36.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,2.0,3.0,0.0,0.0,10.0,1.0,0.0,0.0
2,Released,30.0,0.0,5.0,2.0,2.0,33.0,36.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,2.0,3.0,0.0,0.0,10.0,1.0,0.0,0.0
2,Released,30.0,0.0,5.0,2.0,2.0,33.0,36.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()